** Tugas Besar 2 Intelijensi Buatan **

** Prediksi Income per Tahun **
** Website http://predicio.herokuapp.com/ **

** Anggota Kelompok: **
- Devin Alvaro / 13515062
- Stevanno Hero Leadervand / 13515082
- Rizki Ihza / 13515104
- Gianfranco Fertino Hwandiano / 13515118

# Libraries

In [ ]:
import pandas as pd
import numpy as np

from sklearn import preprocessing, neighbors, tree
from sklearn.externals import joblib
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

%matplotlib inline

# Membaca dataset

In [ ]:
train_df = pd.read_csv("data/CencusIncome.data.txt", header = None)
temp = pd.read_csv("data/CencusIncome.data.txt", header = None)
# name columns
train_df = train_df.rename(columns={0: 'age', 1: 'workclass', 2: 'fnlwgt', 3: 'education', 4: 'education-num', 5: 'marital-status', 6: 'occupation',7: 'relationship', 8: 'race',9: 'sex', 10: 'capital-gain', 11: 'capital-loss', 12: 'hours-per-week', 13: 'native-country', 14: 'label'})
temp = temp.rename(columns={0: 'age', 1: 'workclass', 2: 'fnlwgt', 3: 'education', 4: 'education-num', 5: 'marital-status', 6: 'occupation',7: 'relationship', 8: 'race',9: 'sex', 10: 'capital-gain', 11: 'capital-loss', 12: 'hours-per-week', 13: 'native-country', 14: 'label'})

# Preprocessing

## Feature Selection

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

le = preprocessing.LabelEncoder()

for column in temp:
    le.fit(temp[column])
    temp[column] = le.transform(temp[column])

y = np.array(temp['label'])
x = np.array(temp.drop(['label'], 1))

#feature extraction
model = LogisticRegression()
rfe = RFE(model, 1)
fit = rfe.fit(x, y)
print("Feature Ranking: ")
print(fit.ranking_)

Feature selection merupakan metode untuk memilih subset atribut-atribut/features dari dataset yang dirasa penting saja. Manfaatnya diantara lain yaitu meningkatkan akurasi model dan mempercepat proses training karena data menjadi lebih sedikit.

Metode feature selection yang kelompok kami gunakan yaitu Recursive Feature Elemination (RFE). Metode ini mengurutkan atribut-atribut (ranking) dari urutan 1 (paling penting) hingga seterusnya (semakin tidak penting). 

Dari informasi di atas, dapat disimpulkan bahwa fnlwgt (atribut nomor 3 dari kiri) merupakan atribut yang paling tidak penting (urutan terakhir yaitu 14) jika dibandingkan dengan atribut-atribut lainnya. Berdasarkan hal tersebut, kami memilih untuk tidak mengikutkan atribut fnlwgt pada training model kami.

In [ ]:
# remove 'fnlwgt' column
train_df = train_df.drop(['fnlwgt'], axis=1)

** Missing Value Treatment **

Dari dataset, kami melihat bahwa ada sekitar 4000 baris data yang mengandung missing value, ditandai dengan tanda ?. 

Kami telah melakukan berbagai eksplorasi tentang bagaimana menghandle hal tersebut. 

Kami telah mencoba menghapus data yang bersangkutan dan juga mengganti value ? dengan value modus ataupun rata2 dari atribut yang bersangkutan. 

Namun, semua hal tersebut tidak meningkatkan akurasi dari model kami, malah justru menurunkan akurasi. Jadi, kami memutuskan untuk membiarkan value tersebut.

## Encoding data dengan One-Hot Encoding

In [ ]:
le = preprocessing.LabelEncoder()

le.fit(train_df['label'])
train_df['label'] = le.transform(train_df['label'])

train_df = pd.get_dummies(train_df)

y = np.array(train_df['label'])
x = np.array(train_df.drop(['label'], 1))

** One-Hot Encoding **

Pada dataset yang diberikan, terdapat nilai-nilai berupa string (bukan angka), sehingga harus dilakukan pemrosesan (*encoding*) agar nilai-nilai string tersebut menjadi angka yang dapat dikalkulasi oleh algoritma-algoritma *learning* menjadi model.

Untuk *preprocessing* dataset ini, kami memilih *one-hot encoding*, yaitu *preprocessing* yang mentransformasi nilai-nilai pada setiap kolom bernilai string menjadi kolomnya masing-masing. Kemudian, pada tiap kolom baru ini terdapat nilai biner (1 atau 0) yang menandakan suatu data berada dalam kategori bernilai demikian atau bukan.

Sebagai contoh, terdapat kolom Sex dengan nilai Male dan Female. Kemudian, kolom Sex ini ditransformasi menjadi 2 kolom, yaitu kolom Male dan Female. Untuk suatu data bernilai Male, maka nilai di kolom Male adalah 1 dan di kolom Female adalah 0. Berlaku pula kebalikannya untuk data bernilai Female.

Kami memilih *one-hot encoding* karena nilai-nilai berbentuk string pada dataset ini tidak tepat bila diubah menjadi angka. Misalnya saja bila pada kolom Race, nilai Asian menjadi 0, White menjadi 1, dan Black menjadi 2, menjadi tidak benar karena menandakan suatu ras bernilai lebih tinggi atau rendah dari ras lain.

*One-hot encoding* ini memang memiliki kelemahan yaitu menambah jumlah kolom secara signifikan sehingga meningkatkan kompleksitas algoritma *learning* yang ada, namun masih dapat ditoleransi.

## Eksperimen untuk mendapatkan model terbaik

### Naive Bayes

In [ ]:
gnb = GaussianNB()

score = cross_val_score(gnb, x, y, cv=10)

In [ ]:
for i in range(10):
    print("Fold-" + str(i + 1) + ":", "%0.6f" % score[i])

print()

print("Mean: %0.6f" % score.mean())
print("Accuration: %0.6f (+/- %0.6f)" % (score.mean(), score.std() * 2))

### Decision Tree

In [ ]:
ID3learn = tree.DecisionTreeClassifier(criterion="entropy")

score = cross_val_score(ID3learn, x, y, cv=10)

In [ ]:
for i in range(10):
    print("Fold-" + str(i + 1) + ":", "%0.6f" % score[i])

print()

print("Mean: %0.6f" % score.mean())
print("Accuration: %0.6f (+/- %0.6f)" % (score.mean(), score.std() * 2))

### k-Nearest Neighbors

In [ ]:
n_neighbors = 61

KNNlearn = neighbors.KNeighborsClassifier(n_neighbors, weights='uniform')

score = cross_val_score(KNNlearn, x, y, cv=10)

In [ ]:
for i in range(10):
    print("Fold-" + str(i + 1) + ":", "%0.6f" % score[i])

print()

print("Mean: %0.6f" % score.mean())
print("Accuration: %0.6f (+/- %0.6f)" % (score.mean(), score.std() * 2))

### Multilayer Perceptron

In [ ]:
MLPlearn = MLPClassifier(solver='lbfgs',hidden_layer_sizes=(5, 2))

score = cross_val_score(MLPlearn, x, y, cv=10)

In [ ]:
for i in range(10):
    print("Fold-" + str(i + 1) + ":", "%0.6f" % score[i])

print()

print("Mean: %0.6f" % score.mean())
print("Accuration: %0.2f (+/- %0.6f)" % (score.mean(), score.std() * 2))

** Memilih model terbaik **

setelah dilakukan percobaan learning dengan beberapa algoritma yaitu:

- Naive Bayes
- Decision Tree Learning
- K-Nearest neighbours
- Multilayer Perceptron

didapat model yang memiliki akurasi tertinggi untuk dataset ini adalah model
K-Nearest neighbours. Oleh karena itu dipilih model K-Nearest neighbour untuk
digunakan dalam perhitungan selanjutnya

In [ ]:
KNNlearn = neighbors.KNeighborsClassifier(n_neighbors, weights='uniform')
KNNlearn.fit(x, y)

## Menyimpan model

In [ ]:
joblib.dump(KNNlearn, 'model/best.pkl')
joblib.dump(KNNlearn, '../webapp/model/best.pkl')

## *Loading* model

In [ ]:
KNNlearn = joblib.load('model/best.pkl')

## Evaluasi dan prediksi dengan model terpilih

### Membaca test dataset

In [ ]:
test_df = pd.read_csv("data/CencusIncome.test.txt", header=None, skiprows=1)

# name columns
test_df = test_df.rename(columns={0: 'age', 1: 'workclass', 2: 'fnlwgt', 3: 'education', 4: 'education-num', 5: 'marital-status', 6: 'occupation',7: 'relationship', 8: 'race',9: 'sex', 10: 'capital-gain', 11: 'capital-loss', 12: 'hours-per-week', 13: 'native-country', 14: 'label'})

# remove 'fnlwgt' column
test_df = test_df.drop(['fnlwgt'], axis=1)

test_df.head(10)

### *Preprocessing* test dataset

In [ ]:
le = preprocessing.LabelEncoder()

le.fit(test_df['label'])
test_df['label'] = le.transform(test_df['label'])

In [ ]:
test_df = pd.get_dummies(test_df)

missing_columns = set(train_df.columns) - set(test_df.columns)
for column in missing_columns:
    test_df[column] = 0

y = np.array(test_df['label'])
x = np.array(test_df.drop(['label'], 1))

print(np.shape(y))
print(np.shape(x))

test_df.head(10)

### Hasil prediksi

In [ ]:
score = KNNlearn.score(x, y)
print("Accuracy: ", score * 100 ,"%")

y_pred = KNNlearn.predict(x)

print("Confusion Matrix: ")
print(confusion_matrix(y, y_pred))